In [1]:
# prepare the datasets
import pandas as pd

In [2]:
tob_data = pd.read_csv('tob_domain_only.csv')

tob_data.head(10)

,index,user,question,question_class,question_cluster,answer_v1,answer_v2,v1_x_v2
0,3,100,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
1,4,100,what is structural testing,whQuestion,23,Structural testing (or White-Box testing) is a...,Structural testing (or White-Box testing) is a...,1.000000
2,5,100,what is a software technique,whQuestion,83,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859
3,6,100,what is system testing,whQuestion,29,System testing is a level of software testing ...,System testing is a level of software testing ...,1.000000
4,7,100,what is unit testing,whQuestion,136,Unit testing verifies the functioning in isola...,Unit testing verifies the functioning in isola...,1.000000
5,8,100,what is integration testing,whQuestion,263,Integration testing is the process of verifyin...,Integration testing is the process of verifyin...,1.000000
6,9,100,Can you give me an example of equivalence clas...,Clarify,213,Here is an example of an application of equiva...,"Sorry, I didn't understand your question, coul...",0.045760
7,10,100,How do I apply equivalence class partitioning,whQuestion,188,The equivalence partitioning criterion divides...,"Sorry, I didn't quite catch that",0.057074
8,11,100,What is verification,whQuestion,126,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859
9,12,100,what is validation,whQuestion,134,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",0.269803


In [3]:
gpt_data = pd.read_csv('gpt_answers_sample.csv')

gpt_data.head(10)

,index,cluster,question,answer:gpt-3.5-turbo-16k
0,281,118,what is software testing,Software testing is the process of evaluating ...
1,1406,118,what is software testing,Software testing is the process of evaluating ...
2,794,118,what is software testing,Software testing is the process of evaluating ...
3,2739,118,What Is software testing,Software testing is the process of evaluating ...
4,1728,118,what is software testing,Software testing is the process of evaluating ...
5,2833,118,What is software test,Software testing is a process of evaluating a ...
6,2738,118,What Is software test,Software testing is the process of evaluating ...
7,348,118,What is software testing,Software testing is a process used to evaluate...
8,2194,162,What is a test case?,"A test case is a set of conditions or inputs, ..."
9,2493,162,i want to know what is a test case,A test case refers to a detailed and specific ...


In [4]:
merged = gpt_data.merge(tob_data, on = 'index')
merged

,index,cluster,question_x,answer:gpt-3.5-turbo-16k,user,question_y,question_class,question_cluster,answer_v1,answer_v2,v1_x_v2
0,281,118,what is software testing,Software testing is the process of evaluating ...,114,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
1,1406,118,what is software testing,Software testing is the process of evaluating ...,171,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
2,794,118,what is software testing,Software testing is the process of evaluating ...,146,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
3,2739,118,What Is software testing,Software testing is the process of evaluating ...,94,What Is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
4,1728,118,what is software testing,Software testing is the process of evaluating ...,216,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
...,...,...,...,...,...,...,...,...,...,...,...
240,1585,266,I need to learn about software design,Software design refers to the process of creat...,196,I need to learn about software design,nAnswer,266,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",0.269803
241,1749,169,How to identify a directrix?,To identify the directrix of a conic section (...,21,How to identify a directrix?,whQuestion,169,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",0.337060
242,628,244,what is orcle,Oracle is a multinational technology company t...,134,what is orcle,whQuestion,244,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859
243,741,128,what is incosistence defect,An inconsistent defect refers to a defect in a...,145,what is incosistence defect,whQuestion,128,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",0.827148


In [5]:
merged = merged.drop(['question_y', 'question_class', 'v1_x_v2', 'question_cluster'], axis = 1)

merged

,index,cluster,question_x,answer:gpt-3.5-turbo-16k,user,answer_v1,answer_v2
0,281,118,what is software testing,Software testing is the process of evaluating ...,114,Software testing is an activity of software en...,Software testing is an activity of software en...
1,1406,118,what is software testing,Software testing is the process of evaluating ...,171,Software testing is an activity of software en...,Software testing is an activity of software en...
2,794,118,what is software testing,Software testing is the process of evaluating ...,146,Software testing is an activity of software en...,Software testing is an activity of software en...
3,2739,118,What Is software testing,Software testing is the process of evaluating ...,94,Software testing is an activity of software en...,Software testing is an activity of software en...
4,1728,118,what is software testing,Software testing is the process of evaluating ...,216,Software testing is an activity of software en...,Software testing is an activity of software en...
...,...,...,...,...,...,...,...
240,1585,266,I need to learn about software design,Software design refers to the process of creat...,196,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that"
241,1749,169,How to identify a directrix?,To identify the directrix of a conic section (...,21,I don't know how to respond this,"Sorry, I didn't understand your question, coul..."
242,628,244,what is orcle,Oracle is a multinational technology company t...,134,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean
243,741,128,what is incosistence defect,An inconsistent defect refers to a defect in a...,145,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d..."


In [6]:
merged = merged[['index', 'user', 'cluster', 'question_x', 'answer_v1', 'answer_v2', 'answer:gpt-3.5-turbo-16k']]
merged

,index,user,cluster,question_x,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k
0,281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
1,1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
2,794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
3,2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
4,1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
...,...,...,...,...,...,...,...
240,1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...
241,1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...
242,628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...
243,741,145,128,what is incosistence defect,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",An inconsistent defect refers to a defect in a...


In [7]:
merged = merged.rename({'question_x': 'question'}, axis=1)
merged

,index,user,cluster,question,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k
0,281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
1,1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
2,794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
3,2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
4,1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...
...,...,...,...,...,...,...,...
240,1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...
241,1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...
242,628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...
243,741,145,128,what is incosistence defect,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",An inconsistent defect refers to a defect in a...


In [8]:
from sentence_transformers import SentenceTransformer, util

encoder = SentenceTransformer('all-mpnet-base-v2')

def compare_answers(base, other):
    base_embedings = encoder.encode(base)
    other_embedings = encoder.encode(other)

    cosine_similarity = util.cos_sim(base_embedings, other_embedings)

    return cosine_similarity.item()

In [9]:
merged['v1_v2'] = merged.apply(lambda r: compare_answers(r['answer_v1'], r['answer_v2']), axis=1)

merged

,index,user,cluster,question,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k,v1_v2
0,281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822
1,1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822
2,794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822
3,2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822
4,1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822
...,...,...,...,...,...,...,...,...
240,1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...,0.269803
241,1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...,0.337060
242,628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...,0.325859
243,741,145,128,what is incosistence defect,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",An inconsistent defect refers to a defect in a...,0.827148


In [10]:
merged['v1_gpt'] = merged.apply(lambda r: compare_answers(r['answer_v1'], r['answer:gpt-3.5-turbo-16k']), axis=1)

merged

,index,user,cluster,question,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k,v1_v2,v1_gpt
0,281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.913018
1,1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.915532
2,794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.880498
3,2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.938825
4,1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.897649
...,...,...,...,...,...,...,...,...,...
240,1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...,0.269803,0.088561
241,1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...,0.337060,-0.051637
242,628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...,0.325859,0.036053
243,741,145,128,what is incosistence defect,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",An inconsistent defect refers to a defect in a...,0.827148,0.665195


In [11]:
merged['v2_gpt'] = merged.apply(lambda r: compare_answers(r['answer_v2'], r['answer:gpt-3.5-turbo-16k']), axis=1)

merged

,index,user,cluster,question,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k,v1_v2,v1_gpt,v2_gpt
0,281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.913018,0.914319
1,1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.915532,0.914273
2,794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.880498,0.879256
3,2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.938825,0.939094
4,1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.897649,0.899633
...,...,...,...,...,...,...,...,...,...,...
240,1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...,0.269803,0.088561,0.009492
241,1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...,0.337060,-0.051637,-0.021211
242,628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...,0.325859,0.036053,-0.013625
243,741,145,128,what is incosistence defect,A software defect or bug is a condition in a s...,"Fault is an incorrect step, process, or data d...",An inconsistent defect refers to a defect in a...,0.827148,0.665195,0.599926


In [12]:
merged = merged.set_index('index')

In [13]:
merged

,user,cluster,question,answer_v1,answer_v2,answer:gpt-3.5-turbo-16k,v1_v2,v1_gpt,v2_gpt
index,,,,,,,,,
281,114,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.913018,0.914319
1406,171,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.915532,0.914273
794,146,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.880498,0.879256
2739,94,118,What Is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.938825,0.939094
1728,216,118,what is software testing,Software testing is an activity of software en...,Software testing is an activity of software en...,Software testing is the process of evaluating ...,0.998822,0.897649,0.899633
...,...,...,...,...,...,...,...,...,...
1585,196,266,I need to learn about software design,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",Software design refers to the process of creat...,0.269803,0.088561,0.009492
1749,21,169,How to identify a directrix?,I don't know how to respond this,"Sorry, I didn't understand your question, coul...",To identify the directrix of a conic section (...,0.337060,-0.051637,-0.021211
628,134,244,what is orcle,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,Oracle is a multinational technology company t...,0.325859,0.036053,-0.013625


In [14]:
merged.to_csv('./similarity_sample_dataset.csv')

In [16]:
merged.loc[1710]

user                                                                      213
cluster                                                                   220
question                                                              Quality
answer_v1                   Sorry, I still can't answer that.. Could you a...
answer_v2                                                                  Hi
answer:gpt-3.5-turbo-16k    Quality refers to the characteristics or featu...
v1_v2                                                                0.117703
v1_gpt                                                               0.106838
v2_gpt                                                               0.067174
Name: 1710, dtype: object